[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Unfettered-b/Asgard_Protein_annotation/blob/main/search_cds.ipynb)

# 🔍 CDS Search and Mapping Notebook

This notebook allows you to:
- Search for coding sequences (CDS) by ID or keyword  
- Retrieve corresponding protein sequences and headers  
- Map each CDS to its genome and organism name using metadata from the repository  
- Explore protein and genome relationships interactively

📁 **Repository:** [Asgard_Protein_annotation](https://github.com/Unfettered-b/Asgard_Protein_annotation)


In [ ]:
# ============================================
# 🔧 Setup + Interactive CDS Search Tool in Colab
# ============================================

print("🔧 Setting up... please wait a few seconds for the CDS Search Tool to load 👇")

# --- Clone and prepare repo ---
!apt-get -qq install git-lfs
!git clone https://github.com/Unfettered-b/Asgard_Protein_annotation.git
%cd Asgard_Protein_annotation
!git lfs install
!git lfs pull
!ls -lh data/


In [ ]:
# --- 🔍 CDS Search Tool for Asgard Proteins ---
import pandas as pd
from IPython.display import display, HTML
import ipywidgets as widgets

# --- User interface ---
completeness = "50" # @param ["50", "70", "80", "90", "95"],


search_term = "tubulin" #@param {type:"string"}

# search_button = widgets.Button(
#     description='🔍 Search',
#     button_style='success',
#     layout=widgets.Layout(width='100px')
# )

output = widgets.Output()

def run_search(change=None):
    output.clear_output()
    with output:
        completeness_level = completeness.value
        cds_file = f"data/Proteins_genomes_cp{completeness_level}.csv"

        try:
            cds_db = pd.read_csv(cds_file)
        except FileNotFoundError:
            print(f"❌ File not found: {cds_file}")
            print("Please ensure the CSV is in the 'data/' folder or mount Google Drive if needed.")
            return

        print(f"✅ Loaded database with {len(cds_db)} entries.")
        print(f"🧬 Unique species in database: {cds_db['organism_name'].nunique()}")

        term = search_box.value.strip()
        if not term:
            print("⚠️ Please enter a search term.")
            return

        results = cds_db[cds_db['header'].str.contains(term, case=False, na=False)]
        print(f"🔎 Found {len(results)} results for search term: '{term}'")

        if len(results) > 0:
            display(HTML(results.head(10).to_html(index=False)))
        else:
            print("No matches found.")

# --- Hook up widgets ---
search_box.on_submit(run_search)
search_button.on_click(run_search)

# --- Display interface ---
ui = widgets.HBox([completeness, search_box, search_button])
display(HTML("<h3>🧬 CDS Search Tool for Asgard Proteins</h3>"))
display(ui, output)
